In [ ]:
import brian2 as b
from brian2.units import (amp, cm, ms, mV, mvolt, nS, ohm, pA, siemens, uF, um,
                          uS)
from dendrify import Dendrite, NeuronModel, Soma


# Simulation settings
INTEGRATION_METHOD = 'rk2' # options: 'euler', 'exponential_euler', 'rk2', 'rk4', 'heun'
b.prefs.codegen.target = 'numpy'


# Model params
GL = 60*uS/(cm**2)  # specific resistance
CM = 1*uF/cm**2    # specific leakage conductance
RI = 200*ohm*cm    # intracellular resistivity (axial resistance)
EL = -65*mV    # resting potential

# Input params
I_INJ_SOMA = 43*pA
I_INJ_DEND = 53*pA
T_START = 20*ms
T_INJ = 500*ms
T_END = 120*ms

# Plotting params
params = {"font.family": "Arial",
          "legend.fontsize": 9,
          "legend.handlelength": 1.5,
          "legend.edgecolor": 'inherit',
          "legend.columnspacing": 0.8,
          "legend.handletextpad": 0.5,
          "axes.labelsize": 9,
          "axes.spines.right": False,
          "axes.spines.top": False,
          "axes.edgecolor": "#d3d3d3",
          "axes.titlesize": 9,
          "xtick.color": '#d3d3d3',
          "xtick.labelcolor": 'black',
          "xtick.labelsize": 9,
          "ytick.color": '#d3d3d3',
          "ytick.labelcolor": 'black',
          "xtick.labelsize": 9,
          'mathtext.default': 'regular',
          'lines.markersize': 3,
          'lines.linewidth': 1.25,
          'grid.linestyle': ":",
          'grid.color': "#d3d3d3",
          'text.antialiased': True,
          'lines.antialiased': True,
          'figure.titlesize': 10,
          'figure.dpi': 110 
          }

b.rcParams.update(params)

In [ ]:
# Stock Brian
morpho = b.Cylinder(length=25*um, diameter=25*um, n=1)
morpho.dendrite = b.Cylinder(length=300*um, diameter=1*um,  n=3)

eqs = '''
Im = GL * (EL - v) : amp/meter**2
I : amp (point current)
'''

spatial_neuron = b.SpatialNeuron(morphology=morpho, model=eqs, Cm=CM, Ri=RI,
                                 threshold='v > -40*mV', reset='v = -50*mV',
                                 refractory=3*ms, threshold_location=0,
                                 method=INTEGRATION_METHOD)

# Initialize voltage
spatial_neuron.v = EL

# Set monitors
M_soma = b.StateMonitor(spatial_neuron, 'v', record=[0])
M_dend0 = b.StateMonitor(spatial_neuron.dendrite[0], 'v', record=True)
M_dend1 = b.StateMonitor(spatial_neuron.dendrite[1], 'v', record=True)
M_dend2 = b.StateMonitor(spatial_neuron.dendrite[2], 'v', record=True)

In [ ]:
# Brian + dendrify
soma = Soma('soma', model='leakyIF', length=25*um, diameter=25*um)
dend0 = Dendrite('dend0', length=100*um, diameter=1*um)
dend1 = Dendrite('dend1', length=100*um, diameter=1*um)
dend2 = Dendrite('dend2', length=100*um, diameter=1*um)
dend2.synapse('AMPA', pre='source', g=0.8*nS,  t_decay=2*ms)


connections = [(soma, dend0), (dend0, dend1), (dend1, dend2)]
model = NeuronModel(connections, cm=CM, gl=GL, v_rest=EL, r_axial=RI)
dend_neuron = b.NeuronGroup(N=1, model=model.equations, method=INTEGRATION_METHOD,
                            threshold='V_soma > -40*mV', reset='V_soma = -50*mV',
                            refractory=3*ms, namespace=model.parameters)
# Initialize variables
model.link(dend_neuron)

# Set monitors
M = b.StateMonitor(dend_neuron, ["V_soma", "V_dend0", "V_dend1", "V_dend2"],
                   record=True)

In [ ]:
model.as_graph()

In [ ]:
# Run simulations with somatic current injections
b.store()
for DT in b.linspace(0.025, 0.425, 17):
    print(f"Running simulation with dt = {DT:.3f} ms")
    b.restore()
    b.defaultclock.dt = DT * ms

    # run few ms without input
    b.run(T_START)

    # add a half-second square pulse
    spatial_neuron.I[0] = I_INJ_SOMA
    dend_neuron.I_ext_soma = I_INJ_SOMA
    b.run(T_INJ)

    # remove input current and run until v reaches rest
    spatial_neuron.I[0] = 0*amp
    dend_neuron.I_ext_soma = 0*amp
    b.run(T_END)

    # Plotting
    time = M.t/ms

    # Dendrify:
    Vs = M.V_soma[0]/mV
    Vd0 = M.V_dend0[0]/mV
    Vd1 = M.V_dend1[0]/mV
    Vd2 = M.V_dend2[0]/mV

    # SpatialNeuron
    vs = M_soma[0].v/mV
    vd0 = M_dend0[0].v/mV
    vd1 = M_dend1[0].v/mV
    vd2 = M_dend2[0].v/mV

    fig, ((ax0, ax1), (ax2, ax3)) = b.subplots(2, 2, sharex=True, sharey=True)
    fig.suptitle(
        f'Current injected at soma\n (dt = {DT:.3f} ms | method = {INTEGRATION_METHOD})')

    ax0.set_title('Soma', fontweight='bold')
    ax0.plot(time, Vs, c='black', label='Dendrify')
    ax0.plot(time, vs, c="crimson", ls="--", label='Brian')
    ax0.legend()
    ax0.set_ylabel('Voltage (mV)')

    ax1.set_title('Dendrite 1')
    ax1.plot(time, Vd0, c='black', label='Dendrify')
    ax1.plot(time, vd0, c="crimson", ls="--", label='Brian')

    ax2.set_title('Dendrite 2')
    ax2.plot(time, Vd1, c='black', label='Dendrify')
    ax2.plot(time, vd1, c="crimson", ls="--", label='Brian')
    ax2.set_ylabel('Voltage (mV)')
    ax2.set_xlabel('Time (ms)')

    ax3.set_title('Dendrite 3')
    ax3.plot(time, Vd2, c='black', label='Dendrify')
    ax3.plot(time, vd2, c="crimson", ls="--", label='Brian')
    ax3.set_xlabel('Time (ms)')

    fig.tight_layout()
    b.show()

In [ ]:
# Run simulations with dendritic current injections
for DT in b.linspace(0.025, 0.425, 17):
    print(f"Running simulation with dt = {DT:.3f} ms")
    b.restore()
    b.defaultclock.dt = DT * ms

    # run few ms without input
    b.run(T_START)

    # add a half-second square pulse
    spatial_neuron.I[3] = I_INJ_DEND
    dend_neuron.I_ext_dend2 = I_INJ_DEND
    b.run(T_INJ)

    # remove input current and run until v reaches rest
    spatial_neuron.I[3] = 0*amp
    dend_neuron.I_ext_dend2 = 0*amp
    b.run(T_END)

    # Plotting
    time = M.t/ms

    # Dendrify:
    Vs = M.V_soma[0]/mV
    Vd0 = M.V_dend0[0]/mV
    Vd1 = M.V_dend1[0]/mV
    Vd2 = M.V_dend2[0]/mV

    # SpatialNeuron
    vs = M_soma[0].v/mV
    vd0 = M_dend0[0].v/mV
    vd1 = M_dend1[0].v/mV
    vd2 = M_dend2[0].v/mV

    fig, ((ax0, ax1), (ax2, ax3)) = b.subplots(2, 2, sharex=True, sharey=True)
    fig.suptitle(
        f'Current injected at dendrite 3\n (dt = {DT:.3f} ms | method = {INTEGRATION_METHOD})')

    ax0.set_title('Soma')
    ax0.plot(time, Vs, c='black', label='Dendrify')
    ax0.plot(time, vs, c="crimson", ls="--", label='Brian')
    ax0.legend()
    ax0.set_ylabel('Voltage (mV)')

    ax1.set_title('Dendrite 1')
    ax1.plot(time, Vd0, c='black', label='Dendrify')
    ax1.plot(time, vd0, c="crimson", ls="--", label='Brian')

    ax2.set_title('Dendrite 2')
    ax2.plot(time, Vd1, c='black', label='Dendrify')
    ax2.plot(time, vd1, c="crimson", ls="--", label='Brian')
    ax2.set_ylabel('Voltage (mV)')
    ax2.set_xlabel('Time (ms)')

    ax3.set_title('Dendrite 3', fontweight='bold')
    ax3.plot(time, Vd2, c='black', label='Dendrify')
    ax3.plot(time, vd2, c="crimson", ls="--", label='Brian')
    ax3.set_xlabel('Time (ms)')

    fig.tight_layout()
    b.show()